# Obatining the Full Dataset of UserMirrorer

In our `UserMirrorer` dataset, the raw data from `MIND` and `MovieLens-1M` datasets are distributed under restrictive licenses and cannot
be included directly.

Therefore, this notebook provides a comprehensive, step-by-step pipeline to load the original archives, execute all necessary preprocessing
operations, and assemble the final UserMirrorer training, and test splits.

To derive the full dataset, just click "run all" to execute all cells.

------

In [1]:
!git clone https://github.com/UserMirrorer/UserMirrorer

Cloning into 'UserMirrorer'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 63 (delta 17), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (63/63), 77.24 KiB | 7.02 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [2]:
%cd UserMirrorer
!pip install -U datasets tqdm uszipcode sqlalchemy-mate==2.0.0.0

/content/UserMirrorer
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.2 MB/s eta 0:00:00
  Created wheel for atomicwrites: filename=atomicwrites-1.4.1-py2.py3-none-any.whl size=6943 sha256=f5835d08b58afbee4b147fa8fcf02c6cd353d9077cd04bb6fea94f53078bd764
  Stored in directory: /root/.cache/pip/wheels/f7/99/9c/d24e98c35f30eba0c367ad1e7888d396d676abb35fe1e7611c
Successfully built atomicwrites
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account 

In [3]:
! wget https://recodatasets.z20.web.core.windows.net/newsrec/MINDlarge_train.zip
! wget https://recodatasets.z20.web.core.windows.net/newsrec/MINDlarge_dev.zip
! wget https://recodatasets.z20.web.core.windows.net/newsrec/MINDlarge_test.zip
! wget --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-1m.zip

--2025-05-14 01:37:23--  https://recodatasets.z20.web.core.windows.net/newsrec/MINDlarge_train.zip
Resolving recodatasets.z20.web.core.windows.net (recodatasets.z20.web.core.windows.net)... 52.239.172.161
Connecting to recodatasets.z20.web.core.windows.net (recodatasets.z20.web.core.windows.net)|52.239.172.161|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 531360717 (507M) [application/x-zip-compressed]
Saving to: ‘MINDlarge_train.zip’

MINDlarge_train.zip 100%[===================>] 506.74M   107MB/s    in 4.7s    

2025-05-14 01:37:28 (108 MB/s) - ‘MINDlarge_train.zip’ saved [531360717/531360717]

--2025-05-14 01:37:28--  https://recodatasets.z20.web.core.windows.net/newsrec/MINDlarge_dev.zip
Resolving recodatasets.z20.web.core.windows.net (recodatasets.z20.web.core.windows.net)... 52.239.172.161
Connecting to recodatasets.z20.web.core.windows.net (recodatasets.z20.web.core.windows.net)|52.239.172.161|:443... connected.
HTTP request sent, awaiting response..

In [4]:
!unzip MINDlarge_train.zip -d MINDlarge
!unzip MINDlarge_dev.zip -d MINDlarge_dev
!unzip MINDlarge_test.zip -d MINDlarge_test
!mv MINDlarge_dev/behaviors.tsv MINDlarge/behaviors_valid.tsv
!mv MINDlarge_dev/news.tsv MINDlarge/news_valid.tsv
!mv MINDlarge_test/news.tsv MINDlarge/news_test.tsv
!unzip ml-1m.zip

Archive:  MINDlarge_train.zip
  inflating: MINDlarge/entity_embedding.vec  
  inflating: MINDlarge/news.tsv      
  inflating: MINDlarge/relation_embedding.vec  
  inflating: MINDlarge/behaviors.tsv  
Archive:  MINDlarge_dev.zip
  inflating: MINDlarge_dev/behaviors.tsv  
  inflating: MINDlarge_dev/entity_embedding.vec  
  inflating: MINDlarge_dev/news.tsv  
  inflating: MINDlarge_dev/relation_embedding.vec  
Archive:  MINDlarge_test.zip
  inflating: MINDlarge_test/entity_embedding.vec  
  inflating: MINDlarge_test/news.tsv  
  inflating: MINDlarge_test/relation_embedding.vec  
  inflating: MINDlarge_test/behaviors.tsv  
Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [5]:
!python preprocessing/DataProcessor_ML-1M.py --source_path ml-1m --project_path UserM

Files in dataset directory:
- README
- users.dat
- movies.dat
- ratings.dat
Ratings DataFrame:
   UserID  MovieID  Rating  Timestamp
0       1     1193       5  978300760
1       1      661       3  978302109
2       1      914       3  978301968
3       1     3408       4  978300275
4       1     2355       5  978824291

Users DataFrame:
   UserID Gender  Age  Occupation Zip-code
0       1      F    1          10    48067
1       2      M   56          16    70072
2       3      M   25          15    55117
3       4      M   45           7    02460
4       5      M   25          20    55455

Movies DataFrame:
   MovieID                               Title                        Genres
0        1                    Toy Story (1995)   Animation|Children's|Comedy
1        2                      Jumanji (1995)  Adventure|Children's|Fantasy
2        3             Grumpier Old Men (1995)                Comedy|Romance
3        4            Waiting to Exhale (1995)                  Comedy|Dra

In [6]:
!python preprocessing/DataProcessor_MIND.py --source_path MINDlarge --project_path UserM

Files in dataset directory:
- behaviors_valid.tsv
- relation_embedding.vec
- news.tsv
- behaviors.tsv
- news_test.tsv
- entity_embedding.vec
- news_valid.tsv
100% 100000/100000 [00:00<00:00, 170077.94it/s]
100% 100000/100000 [00:00<00:00, 191114.15it/s]
100% 100000/100000 [00:00<00:00, 119756.75it/s]
100% 100000/100000 [00:00<00:00, 146833.78it/s]
100% 100000/100000 [00:00<00:00, 134473.72it/s]
100% 100000/100000 [00:01<00:00, 87552.04it/s]
100% 100000/100000 [00:00<00:00, 165235.79it/s]
100% 100000/100000 [00:00<00:00, 111934.79it/s]
100% 100000/100000 [00:01<00:00, 97068.89it/s]
100% 100000/100000 [00:01<00:00, 97604.76it/s]
100% 100000/100000 [00:00<00:00, 194935.21it/s]
100% 100000/100000 [00:01<00:00, 81697.22it/s]
100% 100000/100000 [00:01<00:00, 79763.91it/s]
100% 100000/100000 [00:01<00:00, 76280.36it/s]
100% 100000/100000 [00:00<00:00, 216302.70it/s]
100% 100000/100000 [00:00<00:00, 222185.82it/s]
100% 100000/100000 [00:01<00:00, 65212.78it/s]
100% 100000/100000 [00:02<00:00, 

In [7]:
import pandas as pd
from usermirrorer.strategy.mind_strategy import MINDMappingStrategy, MINDDataStrategy
from usermirrorer.strategy.ml1m_strategy import ML1MDataStrategy
from usermirrorer.formatter.mapping import MappingStrategy
from usermirrorer.formatter.formatter import DataFormatter
from usermirrorer.generator.template import texts_to_messages, convert_action_list

import random
import numpy as np

random.seed(0)
np.random.rand(0)


## Create Full Training Set and Eval Set

In [8]:
from datasets import load_dataset

dataset = load_dataset("MirrorUser/UserMirrorer", split="train")
train = dataset.to_pandas()

dataset = load_dataset("MirrorUser/UserMirrorer-eval", split="test")
test = dataset.to_pandas()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/711 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/40.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/668 [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.56M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/6400 [00:00<?, ? examples/s]

### Movielens-1M

In [9]:
data_formatter = DataFormatter(
    ds=ML1MDataStrategy("UserM", "ml-1m"),
    mp=MappingStrategy()
)

train_split = train[train["dataset"] == "ml-1m"].copy()
train_split["user_id"] = train_split["user_id"].astype(int)
train_split["item_id"] = train_split["item_id"].astype(int)
train_split["impression_list"] = train_split["impression_list"].apply(lambda x: [int(i) for i in x])

train_result = data_formatter.get_all_details(train_split)
train_result["prompt"] = train_result["text"].apply(lambda x: texts_to_messages(convert_action_list(x)))

train.loc[train_result.index, "prompt"] = train_result["prompt"]
train.loc[train_result.index, "messages_chosen"] = train.loc[train_result.index].apply(lambda x: x["prompt"] + [x["messages_chosen"][-1]], axis=1)
train.loc[train_result.index, "messages_rejected"] = train.loc[train_result.index].apply(lambda x: x["prompt"] + [x["messages_rejected"][-1]], axis=1)

test_split = test[test["dataset"] == "ml-1m"].copy()
test_split["user_id"] = test_split["user_id"].astype(int)
test_split["item_id"] = test_split["item_id"].astype(int)
test_split["impression_list"] = test_split["impression_list"].apply(lambda x: [int(i) for i in x])

test_result = data_formatter.get_all_details(test_split)
test.loc[test_result.index, "text"] = test_result["text"]

100%|██████████| 640/640 [00:03<00:00, 213.14it/s]


### MIND

In [10]:
data_formatter = DataFormatter(
    ds=MINDDataStrategy("UserM", "MIND"),
    mp=MINDMappingStrategy()
)

train_split = train[train["dataset"] == "MIND"].copy()

train_result = data_formatter.get_all_details(train_split)
train_result["prompt"] = train_result["text"].apply(lambda x: texts_to_messages(convert_action_list(x)))

train.loc[train_result.index, "prompt"] = train_result["prompt"]
train.loc[train_result.index, "messages_chosen"] = train.loc[train_result.index].apply(lambda x: x["prompt"] + [x["messages_chosen"][-1]], axis=1)
train.loc[train_result.index, "messages_rejected"] = train.loc[train_result.index].apply(lambda x: x["prompt"] + [x["messages_rejected"][-1]], axis=1)

test_split = test[test["dataset"] == "MIND"].copy()

test_result = data_formatter.get_all_details(test_split)
test.loc[test_result.index, "text"] = test_result["text"]

100%|██████████| 1280/1280 [02:39<00:00,  8.04it/s]


In [11]:
train = train.loc[:, ["dataset", "messages_chosen", "messages_rejected"]]
train.to_json("UserMirrorer-Full.jsonl.gz", orient="records", lines=True, compression="gzip")

test = test.drop(columns=["impression_list"])
test.to_json("UserMirrorer-eval-Full.jsonl.gz", orient="records", lines=True, compression="gzip")


In [13]:
!sha1sum UserMirrorer-Full.jsonl.gz

c6bfedccd2380463323f064222d55be858bd5010  UserMirrorer-Full.jsonl.gz


In [14]:
!sha1sum UserMirrorer-eval-Full.jsonl.gz

3289aa9002774ffdaf326605617780dd8b12f1f5  UserMirrorer-eval-Full.jsonl.gz


In [12]:
from google.colab import files
files.download("UserMirrorer-Full.jsonl.gz")
files.download("UserMirrorer-eval-Full.jsonl.gz")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

The dataset file `UserMirrorer-Full.jsonl.gz` and `UserMirrorer-eval-Full.jsonl.gz` will be downloaded automatically. Or you can doanload it manually in *files*.